# 1. Data Preparation

In [ ]:
# Library To Upload File
from google.colab import files

In [ ]:
#Upload Kaggle API (Kaggle.json)
uploaded = files.upload()

In [ ]:
#Install Kaggle
!pip install -q kaggle
!pip install keras==2.3.0

In [ ]:
# Make A Space For Data
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

## This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
%%time
# Download Dataset
!kaggle datasets download -d kritikseth/fruit-and-vegetable-image-recognition

In [ ]:
%%time
# Unzip dataset
!unzip fruit-and-vegetable-image-recognition.zip -d dataset

# 2. Dataset Overview

In [ ]:
label = [['garlic' 'cucumber' 'potato' 'ginger' 'eggplant' 'pomegranate'
 'sweetpotato' 'cauliflower' 'soy beans' 'grapes' 'beetroot' 'cabbage'
 'bell pepper' 'kiwi' 'mango' 'jalepeno' 'paprika' 'onion' 'chilli pepper'
 'pineapple' 'raddish' 'lettuce' 'watermelon' 'capsicum' 'carrot' 'peas'
 'lemon' 'pear' 'apple' 'sweetcorn' 'spinach' 'corn' 'turnip' 'tomato'
 'banana' 'orange']]


# 3. Building Model

# 4. Plotting the training and validation accuracy

# 5. Test Model

# 5. Saving the model into .TFLite format